# Gromos Tutorial Pipeline

In [1]:
import os, sys
from pygromos.utils import bash
root_dir = os.getcwd()

#if package is not installed and path not set correct - this helps you out :)
sys.path.append(root_dir+"/..")


import pygromos
from pygromos.gromos.gromosPP import GromosPP
from pygromos.gromos.gromosXX import GromosXX

gromosPP_bin = None
gromosXX_bin = None
gromPP = GromosPP(gromosPP_bin)
gromXX = GromosXX(gromosXX_bin)

project_dir = os.path.abspath(os.path.dirname(pygromos.__file__)+"/../examples/example_files/Tutorial_System")
input_dir = project_dir+"/input"

## Build initial files
### generate Topology
#### build single topologies

In [2]:
from pygromos.data.ff import Gromos54A7

topo_dir = bash.make_folder(project_dir+'/a_topo')

## Make Cl-
sequence = "CL-"
solvent = "H2O"
top_cl = topo_dir+"/cl.top"

gromPP.make_top(in_building_block_lib_path=Gromos54A7.mtb,
                in_parameter_lib_path=Gromos54A7.ifp,
                in_sequence=sequence, in_solvent=solvent,out_top_path=top_cl)

## Make Peptide
sequence = "NH3+ VAL TYR ARG LYSH GLN COO-"
solvent = "H2O"
top_peptide = topo_dir+"/peptide.top"

gromPP.make_top(in_building_block_lib_path=Gromos54A7.mtb, in_parameter_lib_path=Gromos54A7.ifp,
                in_sequence=sequence, in_solvent=solvent,out_top_path=top_peptide)


'/home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/a_topo/peptide.top'

#### combine topology

In [3]:
top_system = topo_dir+"/vac_sys.top"
gromPP.com_top(in_topo_paths=[top_peptide, top_cl], topo_multiplier=[1,2], out_top_path=top_system)


'/home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/a_topo/vac_sys.top'

### generate coordinates


In [4]:
coord_dir = bash.make_folder(project_dir+"/b_coord")
in_pdb = input_dir+"/peptide.pdb"
cnf_peptide = coord_dir+"/cnf_vacuum_peptide.cnf"

cnf_peptide = gromPP.pdb2gromos(in_pdb_path=in_pdb, in_top_path=top_peptide, out_cnf_path=cnf_peptide)

#### add hydrogens

In [5]:
cnf_hpeptide = coord_dir+"/vacuum_hpeptide.cnf"
cnf_hpeptide = gromPP.protonate(in_cnf_path=cnf_peptide, in_top_path=top_peptide, out_cnf_path=cnf_hpeptide)

#### cnf to pdb

In [6]:
out_pdb = coord_dir+"/vacuum_hpeptide.pdb"
out_pdb = gromPP.frameout(in_coord_path=cnf_hpeptide, in_top_path=top_peptide, out_file_path=out_pdb,
                               periodic_boundary_condition="v", out_file_format="pdb", time=0)

### energy minimization - Vacuum

In [7]:
from pygromos.data.simulation_parameters_templates import template_emin_vac
from pygromos.files.gromos_system import gromos_system

out_prefix = "vacuum_emin"
vacuum_emin_dir = bash.make_folder(project_dir+"/c_"+out_prefix)
os.chdir(vacuum_emin_dir)

grom_system = gromos_system.Gromos_System(work_folder=vacuum_emin_dir,
                                          system_name="in_"+out_prefix,
                                          in_top_path=top_peptide,
                                          in_cnf_path=cnf_hpeptide,
                                          in_imd_path=template_emin_vac)

grom_system.adapt_imd()
#del grom_system.imd.POSITIONRES
grom_system.imd.BOUNDCOND.NTB = 0
grom_system.write_files()

out_emin_vacuum = vacuum_emin_dir + "/" + out_prefix
gromXX.md_run(in_imd_path=grom_system.imd.path,
              in_topo_path=grom_system.top.path,
              in_coord_path=grom_system.cnf.path,
              out_prefix=out_emin_vacuum)
cnf_emin_vacuum = out_emin_vacuum+".cnf"
cnf_emin_vacuum

'/home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/c_vacuum_emin/vacuum_emin.cnf'

## Solvatistation and Solvent Energy Minimization
### build box system

In [8]:
from pygromos.data.solvent_coordinates import spc
out_prefix = "box"
box_dir = bash.make_folder(project_dir+"/d_"+out_prefix)

cnf_box = gromPP.sim_box(in_top_path=top_peptide, in_cnf_path=cnf_emin_vacuum,in_solvent_cnf_file_path=spc,
                         out_cnf_path=box_dir+"/"+out_prefix+".cnf",
                         periodic_boundary_condition="r", minwall=0.8, threshold=0.23, rotate=True)

out_pdb = box_dir+"/"+out_prefix+".pdb"

out_pdb = gromPP.frameout(in_coord_path=cnf_box, in_top_path=top_peptide, out_file_path=out_pdb,
                               periodic_boundary_condition="r", out_file_format="pdb", include="ALL", time=0)

sim_box @topo /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/a_topo/peptide.top @pbc r @pos /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/c_vacuum_emin/vacuum_emin.cnf @solvent /home/bschroed/Documents/code/pygromos/pygromos/data/solvent_coordinates/spc.cnf @minwall 0.8  @rotate  @thresh 0.23


### Add Ions

In [9]:
out_prefix = "ion"
cnf_ion = gromPP.ion(in_cnf_path=cnf_box,
                     in_top_path=top_peptide,
                     out_cnf_path=box_dir+"/"+out_prefix+".cnf",
                     negative=[2, "CL-"],verbose=True           )

ion @topo /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/a_topo/peptide.top @pos /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/d_box/box.cnf @pbc v @potential 0.8 @mindist 0.8 @negative 2 CL-


### Energy Minimization BOX

In [10]:
from pygromos.data.simulation_parameters_templates import template_emin
from pygromos.files.gromos_system import gromos_system

out_prefix = "box_emin"
box_emin_dir = bash.make_folder(project_dir+"/e_"+out_prefix)
os.chdir(box_emin_dir)

grom_system = gromos_system.Gromos_System(work_folder=box_emin_dir,
                                          system_name="in_"+out_prefix,
                                          in_top_path=top_system,
                                          in_cnf_path=cnf_ion,
                                          in_imd_path=template_emin)

grom_system.adapt_imd()
grom_system.imd.STEP.NSTLIM = 3000
grom_system.imd.PRINTOUT.NTPR = 300
grom_system.write_files()

cnf_reference_position = grom_system.cnf.write_refpos(box_emin_dir+"/"+out_prefix+"_refpos.rpf")
cnf_position_restraint = grom_system.cnf.write_possrespec(box_emin_dir+"/"+out_prefix+"_posres.pos", residues=list(filter(lambda x: x != "SOLV", grom_system.cnf.get_residues())))

out_emin_box = box_emin_dir + "/" + out_prefix
gromXX.md_run(in_imd_path=grom_system.imd.path,
              in_topo_path=grom_system.top.path,
              in_coord_path=grom_system.cnf.path,
              in_refpos_path=cnf_reference_position,
              in_posresspec_path=cnf_position_restraint,
              out_prefix=out_emin_box, verbose=True)

cnf_emin_box =out_emin_box+".cnf"
cnf_emin_box = gromPP.frameout(in_coord_path=cnf_emin_box, in_top_path=top_system, out_file_path=cnf_emin_box,
                               periodic_boundary_condition="r cog", out_file_format="cnf", include="ALL", time=0)

out_pdb = box_emin_dir+"/"+out_prefix+".pdb"
out_pdb = gromPP.frameout(in_coord_path=cnf_emin_box, in_top_path=top_system, out_file_path=out_pdb,
                               periodic_boundary_condition="r", out_file_format="pdb", include="ALL", time=0)

cnf_emin_box

Cnf
POSRES /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/e_box_emin/box_emin_posres.pos
COMMAND:  md @topo /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/a_topo/vac_sys.top @conf /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/d_box/ion.cnf @input /home/bschroed/Documents/code/pygromos/pygromos/data/simulation_parameters_templates/emin.imd @posresspec /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/e_box_emin/box_emin_posres.pos @refpos /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/e_box_emin/box_emin_refpos.rpf @fin /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/e_box_emin/box_emin.cnf > /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/e_box_emin/box_emin.omd



'/home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/e_box_emin/box_emin.cnf'

## Simulation

### Equilibration NVP
 To be implemented!

In [11]:
from pygromos.data.simulation_parameters_templates import template_md_tut as template_md
from pygromos.files.gromos_system import gromos_system

out_prefix = "eq_NVP"
eq_NVP_dir = bash.make_folder(project_dir+"/f_"+out_prefix)
os.chdir(eq_NVP_dir)

grom_system = gromos_system.Gromos_System(work_folder=eq_NVP_dir,
                                          system_name="in_"+out_prefix,
                                          in_top_path=top_system,
                                          in_cnf_path=cnf_emin_box,
                                          in_imd_path=template_md)

grom_system.adapt_imd(not_ligand_residues="CL-")
grom_system.imd.STEP.NSTLIM = 1000
grom_system.imd.WRITETRAJ.NTWX = 10
grom_system.imd.WRITETRAJ.NTWE = 10
grom_system.imd.INITIALISE.NTIVEL = 1
grom_system.imd.INITIALISE.NTISHK = 1
grom_system.imd.INITIALISE.NTISHI = 1
grom_system.imd.INITIALISE.NTIRTC = 1

grom_system.imd.randomize_seed()
grom_system.rebase_files()
grom_system.write_files()

out_eq_NVP = eq_NVP_dir + "/" + out_prefix
gromXX.md_run(in_imd_path=grom_system.imd.path,
              in_topo_path=grom_system.top.path,
              in_coord_path=grom_system.cnf.path,
              out_tre=True, out_trc=True,
              out_prefix=out_eq_NVP)

cnf_eq_NVP = out_eq_NVP+".cnf"
cnf_eq_NVP


'/home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/f_eq_NVP/eq_NVP.cnf'

### MD NVP

In [12]:
grom_system


GROMOS SYSTEM: in_eq_NVP
################################################################################
WORKDIR: /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/f_eq_NVP
LAST CHECKPOINT: None

FILES: 
	imd: /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/f_eq_NVP/in_eq_NVP.imd
	top: /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/f_eq_NVP/in_eq_NVP.top
	cnf: /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/f_eq_NVP/in_eq_NVP.cnf
GromosXX_bin: None
GromosPP_bin: None
FUTURE PROMISE: False

SYSTEM: 
	PROTEIN:	protein  nresidues: 5 natoms: 71
	Non-LIGANDS:	['CL-']  nmolecules: 0  natoms: 2
	SOLVENT:	SOLV  nmolecules: 930  natoms: 2790



In [13]:
from pygromos.data.simulation_parameters_templates import template_md
from pygromos.files.gromos_system import gromos_system

out_prefix = "md"
md_dir = bash.make_folder(project_dir+"/g_"+out_prefix)
os.chdir(md_dir)

grom_system = gromos_system.Gromos_System(work_folder=md_dir,
                                          system_name="in_"+out_prefix,
                                          in_top_path=top_system,
                                          in_cnf_path=cnf_eq_NVP,
                                          in_imd_path=template_md)

grom_system.adapt_imd(not_ligand_residues="CL-")
grom_system.imd.STEP.NSTLIM = 1000
grom_system.imd.WRITETRAJ.NTWX = 10
grom_system.imd.WRITETRAJ.NTWE = 10
grom_system.imd.INITIALISE.NTIVEL = 0

grom_system.rebase_files()
grom_system.write_files()

out_md = md_dir + "/" + out_prefix
gromXX.md_run(in_imd_path=grom_system.imd.path,
              in_topo_path=grom_system.top.path,
              in_coord_path=grom_system.cnf.path,
              out_tre=True, out_trc=True,
              out_prefix=out_md, verbose=True)

cnf_md = out_md+".cnf"
cnf_md

COMMAND:  md @topo /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/in_md.top @conf /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/in_md.cnf @input /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/in_md.imd @fin /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/md.cnf @trc /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/md.trc @tre /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/md.tre > /home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/md.omd



'/home/bschroed/Documents/code/pygromos/examples/example_files/Tutorial_System/g_md/md.cnf'

## Analysis

In [14]:
out_prefix = "ana"
md_dir = bash.make_folder(project_dir+"/h_"+out_prefix)
os.chdir(md_dir)